In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget
import numpy as np
import pandas as pd
import torch
import torchvision
from torch import optim
import matplotlib.pyplot as plt
import torch.nn.functional as F
from sklearn.metrics import pairwise_distances
from torch.optim.lr_scheduler import ReduceLROnPlateau
from tqdm.notebook import tqdm
import os
import sys
import time
import torch.nn as nn
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from stream_generators.market_basket_dataset import MarketBasketDataset, BinaryEncodingTransform, RemoveItemsTransform
from utils.MatplotlibUtils import reduce_dims_and_plot
from network.auto_encoder import AutoEncoder
from losses.knn_loss import KNNLoss
from sklearn.cluster import DBSCAN
from sklearn.mixture import GaussianMixture
from soft_decision_tree.sdt_model import SDT
from sklearn.metrics import davies_bouldin_score

In [2]:
k = 8
tree_depth = 8
device = 'cuda'
dataset_path = r"/mnt/qnap/ekosman/Groceries_dataset.csv"

## Create the market basket dataset and use one-hot encoding for items

In [3]:
dataset = MarketBasketDataset(dataset_path=dataset_path)

In [4]:
model = AutoEncoder(dataset.n_items, 50, 4).train().to(device)
epochs = 500
lr = 5e-3
batch_size = 512
log_every = 5

In [5]:
dataset.transform = torchvision.transforms.Compose([
    RemoveItemsTransform(p=0.5),
    BinaryEncodingTransform(mapping=dataset.items_to_idx),
]
)
dataset.target_transform = torchvision.transforms.Compose([
    BinaryEncodingTransform(mapping=dataset.items_to_idx),
]
)

In [6]:
model.train()
data_iter = torch.utils.data.DataLoader(dataset,
                                     batch_size=batch_size,
                                     shuffle=True,
                                     num_workers=1,
                                     pin_memory=True)
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
scheduler = ReduceLROnPlateau(optimizer, 'min', verbose=True, factor=0.7, threshold=1e-4)
knn_crt = KNNLoss(k=k).to(device)
losses = []
alpha = 10/170
gamma = 2
for epoch in range(epochs):
    total_loss = 0
    for iteration, (batch, target) in enumerate(data_iter):
        batch = batch.to(device)
        target = target.to(device)
        outputs, iterm = model(batch, return_intermidiate=True)
        mse_loss = F.binary_cross_entropy_with_logits(outputs, target, reduction='none')
        mask = torch.ones_like(mse_loss)
        mask[target == 0] = alpha ** gamma
        mask[target == 1] = (1 - alpha) ** gamma
        mse_loss = (mse_loss * mask).sum(dim=-1).mean()
        try:
            knn_loss = knn_crt(iterm)
            if torch.isinf(knn_loss):
                knn_loss = 0
        except ValueError:
            knn_loss = torch.tensor(0)
        loss = mse_loss + knn_loss
        total_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if iteration % log_every == 0:
            print(f"Epoch {epoch} / {epochs} | iteration {iteration} / {len(data_iter)} | Total Loss: {loss.item()} | KNN Loss: {knn_loss.item()} | BCE Loss: {mse_loss.item()}")
    
    scheduler.step(total_loss / (iteration + 1))
    losses.append(total_loss / (iteration + 1))

Epoch 0 / 500 | iteration 0 / 30 | Total Loss: 8.164026260375977 | KNN Loss: 6.224818229675293 | BCE Loss: 1.9392085075378418
Epoch 0 / 500 | iteration 5 / 30 | Total Loss: 8.184050559997559 | KNN Loss: 6.224360942840576 | BCE Loss: 1.9596898555755615
Epoch 0 / 500 | iteration 10 / 30 | Total Loss: 8.180439949035645 | KNN Loss: 6.224224090576172 | BCE Loss: 1.956215739250183
Epoch 0 / 500 | iteration 15 / 30 | Total Loss: 8.135895729064941 | KNN Loss: 6.223940849304199 | BCE Loss: 1.9119547605514526
Epoch 0 / 500 | iteration 20 / 30 | Total Loss: 8.149843215942383 | KNN Loss: 6.2236647605896 | BCE Loss: 1.9261789321899414
Epoch 0 / 500 | iteration 25 / 30 | Total Loss: 8.154428482055664 | KNN Loss: 6.223090648651123 | BCE Loss: 1.931337833404541
Epoch 1 / 500 | iteration 0 / 30 | Total Loss: 8.125213623046875 | KNN Loss: 6.22247838973999 | BCE Loss: 1.9027354717254639
Epoch 1 / 500 | iteration 5 / 30 | Total Loss: 8.126628875732422 | KNN Loss: 6.222614765167236 | BCE Loss: 1.9040144681

Epoch 10 / 500 | iteration 25 / 30 | Total Loss: 6.929956436157227 | KNN Loss: 5.7535014152526855 | BCE Loss: 1.1764552593231201
Epoch 11 / 500 | iteration 0 / 30 | Total Loss: 6.860462665557861 | KNN Loss: 5.719170093536377 | BCE Loss: 1.141292691230774
Epoch 11 / 500 | iteration 5 / 30 | Total Loss: 6.783545970916748 | KNN Loss: 5.627161502838135 | BCE Loss: 1.1563844680786133
Epoch 11 / 500 | iteration 10 / 30 | Total Loss: 6.727011203765869 | KNN Loss: 5.563403606414795 | BCE Loss: 1.1636075973510742
Epoch 11 / 500 | iteration 15 / 30 | Total Loss: 6.586681842803955 | KNN Loss: 5.452396869659424 | BCE Loss: 1.1342850923538208
Epoch 11 / 500 | iteration 20 / 30 | Total Loss: 6.4945807456970215 | KNN Loss: 5.36434268951416 | BCE Loss: 1.1302380561828613
Epoch 11 / 500 | iteration 25 / 30 | Total Loss: 6.422830581665039 | KNN Loss: 5.273220062255859 | BCE Loss: 1.1496105194091797
Epoch 12 / 500 | iteration 0 / 30 | Total Loss: 6.283742427825928 | KNN Loss: 5.15598201751709 | BCE Loss:

Epoch 21 / 500 | iteration 20 / 30 | Total Loss: 3.7727081775665283 | KNN Loss: 2.703932285308838 | BCE Loss: 1.0687758922576904
Epoch 21 / 500 | iteration 25 / 30 | Total Loss: 3.7198593616485596 | KNN Loss: 2.672633171081543 | BCE Loss: 1.0472261905670166
Epoch 22 / 500 | iteration 0 / 30 | Total Loss: 3.7044451236724854 | KNN Loss: 2.654899835586548 | BCE Loss: 1.0495452880859375
Epoch 22 / 500 | iteration 5 / 30 | Total Loss: 3.72906494140625 | KNN Loss: 2.709667921066284 | BCE Loss: 1.0193970203399658
Epoch 22 / 500 | iteration 10 / 30 | Total Loss: 3.7616140842437744 | KNN Loss: 2.7004189491271973 | BCE Loss: 1.0611951351165771
Epoch 22 / 500 | iteration 15 / 30 | Total Loss: 3.775270462036133 | KNN Loss: 2.7181968688964844 | BCE Loss: 1.0570734739303589
Epoch 22 / 500 | iteration 20 / 30 | Total Loss: 3.724064826965332 | KNN Loss: 2.66469407081604 | BCE Loss: 1.059370756149292
Epoch 22 / 500 | iteration 25 / 30 | Total Loss: 3.689032793045044 | KNN Loss: 2.62568736076355 | BCE L

Epoch 32 / 500 | iteration 10 / 30 | Total Loss: 3.6349692344665527 | KNN Loss: 2.5865023136138916 | BCE Loss: 1.0484668016433716
Epoch 32 / 500 | iteration 15 / 30 | Total Loss: 3.709860324859619 | KNN Loss: 2.6696319580078125 | BCE Loss: 1.0402283668518066
Epoch 32 / 500 | iteration 20 / 30 | Total Loss: 3.582477569580078 | KNN Loss: 2.54008150100708 | BCE Loss: 1.0423959493637085
Epoch 32 / 500 | iteration 25 / 30 | Total Loss: 3.575697898864746 | KNN Loss: 2.571176767349243 | BCE Loss: 1.004521131515503
Epoch 33 / 500 | iteration 0 / 30 | Total Loss: 3.658238410949707 | KNN Loss: 2.632403612136841 | BCE Loss: 1.0258347988128662
Epoch 33 / 500 | iteration 5 / 30 | Total Loss: 3.5832250118255615 | KNN Loss: 2.5571794509887695 | BCE Loss: 1.026045560836792
Epoch 33 / 500 | iteration 10 / 30 | Total Loss: 3.619227647781372 | KNN Loss: 2.572521686553955 | BCE Loss: 1.046705961227417
Epoch 33 / 500 | iteration 15 / 30 | Total Loss: 3.5785794258117676 | KNN Loss: 2.559694528579712 | BCE L

Epoch 43 / 500 | iteration 0 / 30 | Total Loss: 3.610761880874634 | KNN Loss: 2.56288743019104 | BCE Loss: 1.0478744506835938
Epoch 43 / 500 | iteration 5 / 30 | Total Loss: 3.5444395542144775 | KNN Loss: 2.53094220161438 | BCE Loss: 1.0134973526000977
Epoch 43 / 500 | iteration 10 / 30 | Total Loss: 3.677459716796875 | KNN Loss: 2.616302013397217 | BCE Loss: 1.0611577033996582
Epoch 43 / 500 | iteration 15 / 30 | Total Loss: 3.6485652923583984 | KNN Loss: 2.6028311252593994 | BCE Loss: 1.0457342863082886
Epoch 43 / 500 | iteration 20 / 30 | Total Loss: 3.5360727310180664 | KNN Loss: 2.542710304260254 | BCE Loss: 0.993362307548523
Epoch 43 / 500 | iteration 25 / 30 | Total Loss: 3.5642385482788086 | KNN Loss: 2.5197808742523193 | BCE Loss: 1.0444576740264893
Epoch 44 / 500 | iteration 0 / 30 | Total Loss: 3.6286654472351074 | KNN Loss: 2.5832479000091553 | BCE Loss: 1.0454174280166626
Epoch 44 / 500 | iteration 5 / 30 | Total Loss: 3.601832866668701 | KNN Loss: 2.5612523555755615 | BCE

...

Epoch 455 / 500 | iteration 25 / 30 | Total Loss: 3.4797115325927734 | KNN Loss: 2.4524600505828857 | BCE Loss: 1.0272513628005981
Epoch 456 / 500 | iteration 0 / 30 | Total Loss: 3.490490436553955 | KNN Loss: 2.4655580520629883 | BCE Loss: 1.0249323844909668
Epoch 456 / 500 | iteration 5 / 30 | Total Loss: 3.450888156890869 | KNN Loss: 2.4461400508880615 | BCE Loss: 1.0047482252120972
Epoch 456 / 500 | iteration 10 / 30 | Total Loss: 3.4608447551727295 | KNN Loss: 2.4407060146331787 | BCE Loss: 1.0201387405395508
Epoch 456 / 500 | iteration 15 / 30 | Total Loss: 3.427189350128174 | KNN Loss: 2.4095282554626465 | BCE Loss: 1.0176609754562378
Epoch 456 / 500 | iteration 20 / 30 | Total Loss: 3.440563201904297 | KNN Loss: 2.4440314769744873 | BCE Loss: 0.9965318441390991
Epoch 456 / 500 | iteration 25 / 30 | Total Loss: 3.4479925632476807 | KNN Loss: 2.4262053966522217 | BCE Loss: 1.021787166595459
Epoch 457 / 500 | iteration 0 / 30 | Total Loss: 3.444863796234131 | KNN Loss: 2.438103914

Epoch 466 / 500 | iteration 10 / 30 | Total Loss: 3.4270777702331543 | KNN Loss: 2.43412709236145 | BCE Loss: 0.9929506778717041
Epoch 466 / 500 | iteration 15 / 30 | Total Loss: 3.4769558906555176 | KNN Loss: 2.4305708408355713 | BCE Loss: 1.0463851690292358
Epoch 466 / 500 | iteration 20 / 30 | Total Loss: 3.5045700073242188 | KNN Loss: 2.4780433177948 | BCE Loss: 1.0265265703201294
Epoch 466 / 500 | iteration 25 / 30 | Total Loss: 3.434802532196045 | KNN Loss: 2.4517154693603516 | BCE Loss: 0.9830871224403381
Epoch 467 / 500 | iteration 0 / 30 | Total Loss: 3.5032684803009033 | KNN Loss: 2.4783828258514404 | BCE Loss: 1.024885654449463
Epoch 467 / 500 | iteration 5 / 30 | Total Loss: 3.4402925968170166 | KNN Loss: 2.4393630027770996 | BCE Loss: 1.000929594039917
Epoch 467 / 500 | iteration 10 / 30 | Total Loss: 3.4974653720855713 | KNN Loss: 2.4704699516296387 | BCE Loss: 1.0269954204559326
Epoch 467 / 500 | iteration 15 / 30 | Total Loss: 3.4298720359802246 | KNN Loss: 2.4396941661

Epoch 476 / 500 | iteration 25 / 30 | Total Loss: 3.4186818599700928 | KNN Loss: 2.420632839202881 | BCE Loss: 0.9980490803718567
Epoch 477 / 500 | iteration 0 / 30 | Total Loss: 3.4992356300354004 | KNN Loss: 2.478489875793457 | BCE Loss: 1.0207457542419434
Epoch 477 / 500 | iteration 5 / 30 | Total Loss: 3.456890821456909 | KNN Loss: 2.46777081489563 | BCE Loss: 0.9891200065612793
Epoch 477 / 500 | iteration 10 / 30 | Total Loss: 3.455470561981201 | KNN Loss: 2.433079719543457 | BCE Loss: 1.0223909616470337
Epoch 477 / 500 | iteration 15 / 30 | Total Loss: 3.4551830291748047 | KNN Loss: 2.440864086151123 | BCE Loss: 1.0143189430236816
Epoch 477 / 500 | iteration 20 / 30 | Total Loss: 3.474964141845703 | KNN Loss: 2.452160120010376 | BCE Loss: 1.0228039026260376
Epoch 477 / 500 | iteration 25 / 30 | Total Loss: 3.4621801376342773 | KNN Loss: 2.4839096069335938 | BCE Loss: 0.9782705307006836
Epoch 478 / 500 | iteration 0 / 30 | Total Loss: 3.4525279998779297 | KNN Loss: 2.4353661537170

Epoch 487 / 500 | iteration 10 / 30 | Total Loss: 3.4130349159240723 | KNN Loss: 2.427854061126709 | BCE Loss: 0.9851807355880737
Epoch 487 / 500 | iteration 15 / 30 | Total Loss: 3.469191312789917 | KNN Loss: 2.467212677001953 | BCE Loss: 1.0019786357879639
Epoch 487 / 500 | iteration 20 / 30 | Total Loss: 3.43693470954895 | KNN Loss: 2.429675340652466 | BCE Loss: 1.0072593688964844
Epoch 487 / 500 | iteration 25 / 30 | Total Loss: 3.4785704612731934 | KNN Loss: 2.4568064212799072 | BCE Loss: 1.0217640399932861
Epoch 488 / 500 | iteration 0 / 30 | Total Loss: 3.4128501415252686 | KNN Loss: 2.4294497966766357 | BCE Loss: 0.983400285243988
Epoch 488 / 500 | iteration 5 / 30 | Total Loss: 3.4198803901672363 | KNN Loss: 2.432467222213745 | BCE Loss: 0.987413227558136
Epoch 488 / 500 | iteration 10 / 30 | Total Loss: 3.455881118774414 | KNN Loss: 2.426189661026001 | BCE Loss: 1.029691457748413
Epoch 488 / 500 | iteration 15 / 30 | Total Loss: 3.426206588745117 | KNN Loss: 2.421798229217529

Epoch   498: reducing learning rate of group 0 to 9.5791e-07.
Epoch 498 / 500 | iteration 0 / 30 | Total Loss: 3.493215560913086 | KNN Loss: 2.460930585861206 | BCE Loss: 1.0322850942611694
Epoch 498 / 500 | iteration 5 / 30 | Total Loss: 3.4536564350128174 | KNN Loss: 2.4292213916778564 | BCE Loss: 1.024435043334961
Epoch 498 / 500 | iteration 10 / 30 | Total Loss: 3.451491594314575 | KNN Loss: 2.449463367462158 | BCE Loss: 1.002028226852417
Epoch 498 / 500 | iteration 15 / 30 | Total Loss: 3.4769890308380127 | KNN Loss: 2.4451828002929688 | BCE Loss: 1.031806230545044
Epoch 498 / 500 | iteration 20 / 30 | Total Loss: 3.489982843399048 | KNN Loss: 2.4779584407806396 | BCE Loss: 1.0120244026184082
Epoch 498 / 500 | iteration 25 / 30 | Total Loss: 3.477921724319458 | KNN Loss: 2.450702428817749 | BCE Loss: 1.027219295501709
Epoch 499 / 500 | iteration 0 / 30 | Total Loss: 3.42901873588562 | KNN Loss: 2.4322667121887207 | BCE Loss: 0.9967520833015442
Epoch 499 / 500 | iteration 5 / 30 | 

In [7]:
outputs, iterm = model(dataset[67][0].unsqueeze(0).to(device), return_intermidiate=True)
print(outputs)
print(outputs.min())
print(outputs.max())

tensor([[ 3.0974,  2.4199,  2.8636,  3.6784,  2.6929,  0.8784,  2.8699,  2.5165,
          2.5702,  2.1222,  2.4783,  2.5003,  1.1700,  2.0119,  1.4261,  1.7646,
          1.8027,  3.2493,  2.1962,  2.4891,  1.9998,  2.9664,  2.0832,  2.3126,
          2.5504,  1.9909,  2.3046,  1.7974,  1.6798,  0.1771, -0.2215,  0.4746,
          0.3704,  1.1309,  1.7533,  1.6911,  1.4606,  2.5366,  0.3809,  1.5309,
          0.8706, -0.5006,  0.1272,  2.5845,  1.7429,  0.9854, -0.0687,  0.3883,
          1.7073,  2.3993,  1.8991,  0.4401,  1.8039,  0.7181, -0.4625,  1.3088,
          0.9047,  1.4864,  1.5912,  2.0448,  0.6664,  0.2319, -0.2534,  1.8781,
          1.6133,  2.0089, -2.0149,  0.5438,  2.2521,  2.2253,  1.9007,  0.7418,
          1.6033,  2.0876,  1.7641,  1.5441,  0.2970,  0.8440,  0.4793,  1.8122,
          0.1397,  0.6163,  1.9695, -0.3058,  0.2172, -0.6535, -2.0342, -0.1630,
          0.7243, -2.3721,  0.7076, -0.0236, -0.5952, -0.6824,  0.4202,  1.4988,
         -0.9907, -0.6045,  

In [8]:
plt.figure()
plt.plot(losses)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
dataset.transform = torchvision.transforms.Compose([
    BinaryEncodingTransform(mapping=dataset.items_to_idx),
]
)
dataset.target_transform = torchvision.transforms.Compose([
    BinaryEncodingTransform(mapping=dataset.items_to_idx),
]
)

In [14]:
dataset_ = [d[0].cpu() for d in dataset]

In [15]:
model = model.eval().cpu()
projections = model.calculate_intermidiate(dataset_)

100%|██████████| 15/15 [00:00<00:00, 91.38it/s]


In [16]:
distances = pairwise_distances(projections)
# distances = np.triu(distances)
distances_f = distances.flatten()

plt.matshow(distances)
plt.colorbar()
plt.figure()
plt.hist(distances_f[distances_f > 0], bins=1000)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Fit DBSCAN and calculate indices


In [17]:
clusters = DBSCAN(eps=0.2, min_samples=80).fit_predict(projections)
# scores = []
# best_score = float('inf')
# clusters = None
# range_ = list(range(5, 20))
# for k in tqdm(range_):
#     y = GaussianMixture(n_components=k).fit_predict(projections)
#     cur_score = davies_bouldin_score(projections, y)
#     scores.append(cur_score)
    
#     if cur_score < best_score:
#         best_score = cur_score
#         clusters = y

In [18]:
perplexity = 100

p = reduce_dims_and_plot(projections[clusters != -1],
                         y=clusters[clusters != -1],
                         title=f'perplexity: {perplexity}',
                         file_name=None,
                         perplexity=perplexity,
                         library='Multicore-TSNE',
                         perform_PCA=False,
                         projected=None,
                         figure_type='2d',
                         show_figure=True,
                         close_figure=False,
                         text=None)    

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [19]:
# from sklearn.tree import DecisionTreeClassifier
# from sklearn import tree
# from sklearn.tree import _tree

In [21]:
tensor_dataset = torch.stack(dataset_)

In [22]:
# clf = DecisionTreeClassifier(max_depth=200, min_samples_leaf=5)
# clf = clf.fit(tensor_dataset[clusters!=-1], clusters[clusters != -1])
# print(clf.score(tensor_dataset[clusters!=-1], clusters[clusters != -1]))
# print(clf.get_depth())

In [23]:
# scores = []
# for min_samples in range(1,50, 1):
#     clf = DecisionTreeClassifier(max_depth=200, min_samples_leaf=min_samples)
#     clf = clf.fit(tensor_dataset[clusters!=-1], clusters[clusters != -1])
#     scores.append(clf.score(tensor_dataset[clusters!=-1], clusters[clusters != -1]))
    
# plt.figure()
# plt.plot(list(range(1,50, 1)), scores)
# plt.show()

In [24]:
def get_rules(tree, feature_names, class_names):
    tree_ = tree.tree_
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]

    paths = []
    path = []
    
    def recurse(node, path, paths):
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]
            p1, p2 = list(path), list(path)
#             p1 += [f"({name} <= {np.round(threshold, 3)})"]
            p1 += [(name, '<=', np.round(threshold, 3))]
            recurse(tree_.children_left[node], p1, paths)
            p2 += [(name, '>', np.round(threshold, 3))]
            recurse(tree_.children_right[node], p2, paths)
        else:
            path += [(tree_.value[node], tree_.n_node_samples[node])]
            paths += [path]
            
    recurse(0, path, paths)

    # sort by samples count
    samples_count = [p[-1][1] for p in paths]
    ii = list(np.argsort(samples_count))
    paths = [paths[i] for i in reversed(ii)]
    
    rules = []
    for path in paths:
        rule = []
        
        for p in path[:-1]:
            rule += [p]
        target = " then "
        if class_names is None:
            target += "response: "+str(np.round(path[-1][0][0][0],3))
        else:
            classes = path[-1][0][0]
            l = np.argmax(classes)
            target += f"class: {class_names[l]} (proba: {np.round(100.0*classes[l]/np.sum(classes),2)}%)"
           
        proba = np.round(100.0*classes[l]/np.sum(classes),2)
        target += f" | based on {path[-1][1]:,} samples"
        rule_wrapper = {'target': target, 'rule': rule, 'proba': proba}
        rules += [rule_wrapper]
        
    return rules

In [25]:
# rules = get_rules(clf, dataset.items, clusters[clusters != -1])

# for rule in rules:
#     n_pos = 0
#     for c,p,v in rule['rule']:
#         if p == '>':
#             n_pos += 1
#     rule['pos'] = n_pos

In [26]:
# plt.figure()
# probs = [r['proba'] for r in rules]
# plt.hist(probs, bins = 100)
# plt.show()

In [27]:
# rules = sorted(rules, key=lambda x:x['pos'])
# rules = [r for r in rules if r['proba'] > 50]
# print(len(rules))

In [28]:
# for i in range(17):
#     r_i = rules[i]
#     print(f"------------- rule {i} length {len(r_i)} -------------")
#     print(r_i)

# Train a Soft-Decision-Tree given the self-labels

## Prepare the dataset

In [29]:
tree_dataset = list(zip(tensor_dataset[clusters!=-1], clusters[clusters != -1]))
batch_size = 512
tree_loader = torch.utils.data.DataLoader(tree_dataset, batch_size=batch_size, shuffle=True)

# Define how we prune the weights of a node

In [30]:
def prune_node(node_weights, factor=1):
    w = node_weights.cpu().detach().numpy()
    mean_ = np.mean(w)
    std_ = np.std(w)
    node_weights[((mean_ - std_ * factor) < node_weights) & (node_weights < (mean_ + std_ * factor))] = 0
    return node_weights

def prune_node_keep(node_weights, keep=4):
    w = node_weights.cpu().detach().numpy()
    throw_idx = np.argsort(abs(w))[:-keep]
    node_weights[throw_idx] = 0
    return node_weights

def prune_tree(tree_, factor):
    new_weights = tree_.inner_nodes.weight.clone()
    for i in range(new_weights.shape[0]):
        res = prune_node_keep(new_weights[i, :], factor)
        new_weights[i, :] = res

    with torch.no_grad():
        tree_.inner_nodes.weight.copy_(new_weights)
        
def sparseness(x):
    s = []
    for i in range(x.shape[0]):
        x_ = x[i, :]
        sp = (len(x_) - torch.norm(x_, 0).item()) / len(x_)
        s.append(sp)
    return np.mean(s)

def compute_regularization_by_level(tree):
    total_reg = 0
    for i in range(tree.inner_nodes.weight.shape[0]):
        cur_level = np.floor(np.log2(i+1))
        node_reg = torch.norm(tree.inner_nodes.weight[i].view(-1), 2)
        total_reg += 2**(-cur_level) * node_reg
    return total_reg

def show_sparseness(tree):
    avg_sp = sparseness(tree.inner_nodes.weight)
    print(f"Average sparseness: {avg_sp}")
    layer = 0
    sps = []
    for i in range(tree.inner_nodes.weight.shape[0]):
        cur_layer = int(np.floor(np.log2(i+1)))
        if cur_layer != layer:
            print(f"layer {layer}: {np.mean(sps)}")
            sps = []
            layer = cur_layer

        x_ = tree.inner_nodes.weight[i, :]
        sp = (len(x_) - torch.norm(x_, 0).item()) / len(x_)
        sps.append(sp)
        
    return avg_sp

## Training configurations

In [31]:
def do_epoch(model, loader, device, log_interval, losses, accs, epoch, iteration):
    model = model.train()
    for batch_idx, (data, target) in enumerate(loader):
        iteration += 1
        data, target = data.to(device), target.to(device)

        output, penalty = tree.forward(data)

        # Loss
        loss_tree = criterion(output, target.view(-1))

        # Penalty
        loss_tree += penalty

        # Sparse regularization
#         fc_params = torch.cat([x.view(-1) for x in tree.inner_nodes.parameters()])
#         regularization = sparsity_lamda * torch.norm(fc_params, 2)
        regularization = sparsity_lamda * compute_regularization_by_level(tree)
        loss = loss_tree

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        losses.append(loss.item())

        pred = output.data.max(1)[1]
        correct = pred.eq(target.view(-1).data).sum()
        accs.append(correct.item() / data.size()[0])

        # Print training status
        if batch_idx % log_interval == 0:
            print(f"Epoch: {epoch:02d} | Batch: {batch_idx:03d} / {len(loader):03d} | Total loss: {loss.item():.3f} | Reg loss: {regularization.item():.3f} | Tree loss: {loss_tree.item():.3f} | Accuracy: {correct.item() / data.size()[0]:03f} | {round((time.time() - start_time) / iteration, 3)} sec/iter")
            
    return iteration


In [32]:
lr = 5e-3
weight_decay = 5e-4
sparsity_lamda = 2e-3
epochs = 100
output_dim = len(set(clusters))
log_interval = 1
use_cuda = device != 'cpu'

In [33]:
tree = SDT(input_dim=tensor_dataset.shape[1], output_dim=len(clusters - 1), depth=tree_depth, lamda=1e-3, use_cuda=use_cuda)
optimizer = torch.optim.Adam(tree.parameters(),
                                 lr=lr,
                                 weight_decay=weight_decay)
criterion = nn.CrossEntropyLoss()
tree = tree.to(device)

In [34]:
losses = []
accs = []
sparsity = []

In [35]:
start_time = time.time()
iteration = 0
for epoch in range(epochs):
    # Training
    avg_sp = show_sparseness(tree)
    sparsity.append(avg_sp)
    iteration = do_epoch(tree, tree_loader, device, log_interval, losses, accs, epoch, iteration)
    
    if epoch % 1 == 0:
        prune_tree(tree, factor=3)
        

Average sparseness: 0.0
layer 0: 0.0
layer 1: 0.0
layer 2: 0.0
layer 3: 0.0
layer 4: 0.0
layer 5: 0.0
layer 6: 0.0
Epoch: 00 | Batch: 000 / 002 | Total loss: 9.605 | Reg loss: 0.009 | Tree loss: 9.605 | Accuracy: 0.000000 | 0.245 sec/iter
Epoch: 00 | Batch: 001 / 002 | Total loss: 9.591 | Reg loss: 0.009 | Tree loss: 9.591 | Accuracy: 0.000000 | 0.216 sec/iter
Average sparseness: 0.9821428571428573
layer 0: 0.9821428571428571
layer 1: 0.9821428571428571
layer 2: 0.9821428571428571
layer 3: 0.9821428571428571
layer 4: 0.9821428571428571
layer 5: 0.9821428571428571
layer 6: 0.982142857142857
Epoch: 01 | Batch: 000 / 002 | Total loss: 9.589 | Reg loss: 0.002 | Tree loss: 9.589 | Accuracy: 0.000000 | 0.221 sec/iter
Epoch: 01 | Batch: 001 / 002 | Total loss: 9.578 | Reg loss: 0.002 | Tree loss: 9.578 | Accuracy: 0.000000 | 0.216 sec/iter
Average sparseness: 0.9821428571428573
layer 0: 0.9821428571428571
layer 1: 0.9821428571428571
layer 2: 0.9821428571428571
layer 3: 0.9821428571428571
laye

Epoch: 17 | Batch: 000 / 002 | Total loss: 9.449 | Reg loss: 0.005 | Tree loss: 9.449 | Accuracy: 0.140625 | 0.214 sec/iter
Epoch: 17 | Batch: 001 / 002 | Total loss: 9.439 | Reg loss: 0.005 | Tree loss: 9.439 | Accuracy: 0.166292 | 0.214 sec/iter
Average sparseness: 0.9821428571428573
layer 0: 0.9821428571428571
layer 1: 0.9821428571428571
layer 2: 0.9821428571428571
layer 3: 0.9821428571428571
layer 4: 0.9821428571428571
layer 5: 0.9821428571428571
layer 6: 0.982142857142857
Epoch: 18 | Batch: 000 / 002 | Total loss: 9.441 | Reg loss: 0.005 | Tree loss: 9.441 | Accuracy: 0.148438 | 0.215 sec/iter
Epoch: 18 | Batch: 001 / 002 | Total loss: 9.429 | Reg loss: 0.005 | Tree loss: 9.429 | Accuracy: 0.184270 | 0.214 sec/iter
Average sparseness: 0.9821428571428573
layer 0: 0.9821428571428571
layer 1: 0.9821428571428571
layer 2: 0.9821428571428571
layer 3: 0.9821428571428571
layer 4: 0.9821428571428571
layer 5: 0.9821428571428571
layer 6: 0.982142857142857
Epoch: 19 | Batch: 000 / 002 | Total

Epoch: 34 | Batch: 000 / 002 | Total loss: 9.307 | Reg loss: 0.006 | Tree loss: 9.307 | Accuracy: 0.306641 | 0.219 sec/iter
Epoch: 34 | Batch: 001 / 002 | Total loss: 9.301 | Reg loss: 0.007 | Tree loss: 9.301 | Accuracy: 0.323596 | 0.218 sec/iter
Average sparseness: 0.9821428571428573
layer 0: 0.9821428571428571
layer 1: 0.9821428571428571
layer 2: 0.9821428571428571
layer 3: 0.9821428571428571
layer 4: 0.9821428571428571
layer 5: 0.9821428571428571
layer 6: 0.982142857142857
Epoch: 35 | Batch: 000 / 002 | Total loss: 9.295 | Reg loss: 0.007 | Tree loss: 9.295 | Accuracy: 0.326172 | 0.219 sec/iter
Epoch: 35 | Batch: 001 / 002 | Total loss: 9.298 | Reg loss: 0.007 | Tree loss: 9.298 | Accuracy: 0.301124 | 0.219 sec/iter
Average sparseness: 0.9821428571428573
layer 0: 0.9821428571428571
layer 1: 0.9821428571428571
layer 2: 0.9821428571428571
layer 3: 0.9821428571428571
layer 4: 0.9821428571428571
layer 5: 0.9821428571428571
layer 6: 0.982142857142857
Epoch: 36 | Batch: 000 / 002 | Total

Epoch: 51 | Batch: 001 / 002 | Total loss: 9.163 | Reg loss: 0.008 | Tree loss: 9.163 | Accuracy: 0.312360 | 0.224 sec/iter
Average sparseness: 0.9821428571428573
layer 0: 0.9821428571428571
layer 1: 0.9821428571428571
layer 2: 0.9821428571428571
layer 3: 0.9821428571428571
layer 4: 0.9821428571428571
layer 5: 0.9821428571428571
layer 6: 0.982142857142857
Epoch: 52 | Batch: 000 / 002 | Total loss: 9.161 | Reg loss: 0.008 | Tree loss: 9.161 | Accuracy: 0.347656 | 0.224 sec/iter
Epoch: 52 | Batch: 001 / 002 | Total loss: 9.164 | Reg loss: 0.008 | Tree loss: 9.164 | Accuracy: 0.276404 | 0.224 sec/iter
Average sparseness: 0.9821428571428573
layer 0: 0.9821428571428571
layer 1: 0.9821428571428571
layer 2: 0.9821428571428571
layer 3: 0.9821428571428571
layer 4: 0.9821428571428571
layer 5: 0.9821428571428571
layer 6: 0.982142857142857
Epoch: 53 | Batch: 000 / 002 | Total loss: 9.159 | Reg loss: 0.008 | Tree loss: 9.159 | Accuracy: 0.320312 | 0.225 sec/iter
Epoch: 53 | Batch: 001 / 002 | Total

Epoch: 68 | Batch: 001 / 002 | Total loss: 9.026 | Reg loss: 0.010 | Tree loss: 9.026 | Accuracy: 0.332584 | 0.229 sec/iter
Average sparseness: 0.9821428571428573
layer 0: 0.9821428571428571
layer 1: 0.9821428571428571
layer 2: 0.9821428571428571
layer 3: 0.9821428571428571
layer 4: 0.9821428571428571
layer 5: 0.9821428571428571
layer 6: 0.982142857142857
Epoch: 69 | Batch: 000 / 002 | Total loss: 9.030 | Reg loss: 0.010 | Tree loss: 9.030 | Accuracy: 0.335938 | 0.23 sec/iter
Epoch: 69 | Batch: 001 / 002 | Total loss: 9.025 | Reg loss: 0.010 | Tree loss: 9.025 | Accuracy: 0.289888 | 0.229 sec/iter
Average sparseness: 0.9821428571428573
layer 0: 0.9821428571428571
layer 1: 0.9821428571428571
layer 2: 0.9821428571428571
layer 3: 0.9821428571428571
layer 4: 0.9821428571428571
layer 5: 0.9821428571428571
layer 6: 0.982142857142857
Epoch: 70 | Batch: 000 / 002 | Total loss: 9.030 | Reg loss: 0.010 | Tree loss: 9.030 | Accuracy: 0.306641 | 0.23 sec/iter
Epoch: 70 | Batch: 001 / 002 | Total l

Epoch: 86 | Batch: 000 / 002 | Total loss: 8.889 | Reg loss: 0.011 | Tree loss: 8.889 | Accuracy: 0.316406 | 0.232 sec/iter
Epoch: 86 | Batch: 001 / 002 | Total loss: 8.884 | Reg loss: 0.011 | Tree loss: 8.884 | Accuracy: 0.312360 | 0.232 sec/iter
Average sparseness: 0.9821428571428573
layer 0: 0.9821428571428571
layer 1: 0.9821428571428571
layer 2: 0.9821428571428571
layer 3: 0.9821428571428571
layer 4: 0.9821428571428571
layer 5: 0.9821428571428571
layer 6: 0.982142857142857
Epoch: 87 | Batch: 000 / 002 | Total loss: 8.886 | Reg loss: 0.011 | Tree loss: 8.886 | Accuracy: 0.306641 | 0.233 sec/iter
Epoch: 87 | Batch: 001 / 002 | Total loss: 8.868 | Reg loss: 0.011 | Tree loss: 8.868 | Accuracy: 0.323596 | 0.232 sec/iter
Average sparseness: 0.9821428571428573
layer 0: 0.9821428571428571
layer 1: 0.9821428571428571
layer 2: 0.9821428571428571
layer 3: 0.9821428571428571
layer 4: 0.9821428571428571
layer 5: 0.9821428571428571
layer 6: 0.982142857142857
Epoch: 88 | Batch: 000 / 002 | Total

In [36]:
plt.figure(figsize=(10, 5))
plt.ylabel("Accuracy")
plt.xlabel('Iteration')
plt.plot(accs, label='Accuracy vs iteration')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [37]:
plt.figure()
plt.ylabel("Loss")
plt.xlabel('Iteration')
plt.plot(losses, label='Loss vs iteration')
plt.yscale("log")
plt.show()

plt.figure()
weights = tree.inner_nodes.weight.cpu().detach().numpy().flatten()
plt.hist(weights, bins=500)
weights_std = np.std(weights)
weights_mean = np.mean(weights)
plt.axvline(weights_mean + weights_std, color='r')
plt.axvline(weights_mean - weights_std, color='r')
plt.title(f"Mean: {weights_mean}   |   STD: {weights_std}")
plt.yscale("log")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Tree Visualization

In [38]:
plt.figure(figsize=(15, 10), dpi=80)
avg_height, root = tree.visualize()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Average height: 8.0


# Extract Rules

# Accumulate samples in the leaves

In [39]:
print(f"Number of patterns: {len(root.get_leaves())}")

Number of patterns: 256


In [40]:
method = 'greedy'

In [41]:
root.clear_leaves_samples()

with torch.no_grad():
    for batch_idx, (data, target) in enumerate(tree_loader):
        root.accumulate_samples(data, method)

/home/eitan.k/.local/lib/python3.6/site-packages/torch/nn/functional.py:1709: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


# Tighten boundaries

In [43]:
attr_names = dataset.items

# print(attr_names)
leaves = root.get_leaves()
sum_comprehensibility = 0
comprehensibilities = []
for pattern_counter, leaf in enumerate(leaves):
    leaf.reset_path()
    leaf.tighten_with_accumulated_samples()
    conds = leaf.get_path_conditions(attr_names)
    print(f"============== Pattern {pattern_counter + 1} ==============")
    comprehensibilities.append(sum([cond.comprehensibility for cond in conds]))
    
print(f"Average comprehensibility: {np.mean(comprehensibilities)}")
print(f"std comprehensibility: {np.std(comprehensibilities)}")
print(f"var comprehensibility: {np.var(comprehensibilities)}")
print(f"minimum comprehensibility: {np.min(comprehensibilities)}")
print(f"maximum comprehensibility: {np.max(comprehensibilities)}")

/home/eitan.k/EntangledExplainableClustering/soft_decision_tree/sdt_model.py:14: RuntimeWarning: divide by zero encountered in log
  return np.log(1 / (1 - x))


============== Pattern 1 ==============
486
============== Pattern 2 ==============
============== Pattern 3 ==============
============== Pattern 4 ==============
147
============== Pattern 5 ==============
20
============== Pattern 6 ==============
============== Pattern 7 ==============
============== Pattern 8 ==============
27
============== Pattern 9 ==============
3
============== Pattern 10 ==============
16
============== Pattern 11 ==============
============== Pattern 12 ==============
3
============== Pattern 13 ==============
============== Pattern 14 ==============
============== Pattern 15 ==============
============== Pattern 16 ==============
============== Pattern 17 ==============
============== Pattern 18 ==============
46
============== Pattern 19 ==============
8
============== Pattern 20 ==============
============== Pattern 21 ==============
============== Pattern 22 ==============
============== Pattern 23 ==============
============== Pattern 24 ==============